# Dutch UMLS to concept table
This notebook describes how to convert a UMLS concept table containing Dutch terms, to a formatted concept table to be used in a tool such as MedCAT. In the second part of this notebook, we add drug names from Dutch SNOMED, because these concepts are not well represented in the Dutch UMLS source vocabularies. A large scale automatic mapping from SNOMED Dutch to UMLS is not possible because there of many-to-mapping mapping.

Requirements:
- MySQL database containing Dutch UMLS terms

For adding Dutch SNOMED drug names:
- Dutch SNOMED concept table
- MySQL database containing SNOMED-US, which is used for mapping SNOMED Dutch -> UMLS

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import json
import re
import os

In [ ]:
# Credentials to connect to UMLS MySQL database
host = '127.0.0.1'
username = 'root'
password = 'test'
port='3306'
database = 'umls'

# Create the connection
connection_string = f'mysql://{username}:{password}@{host}:{port}/{database}'
connection = create_engine(connection_string)

In [ ]:
# Retrieve Dutch UMLS concepts
query = """
SELECT *
FROM umls.mrconso WHERE lat = 'DUT'
"""
df_dutch_umls = pd.read_sql_query(query, con=connection)
df_dutch_umls.head()

## Term type in source
Some source-defined term types are not relevant for our use case. In the next part we will drop those. See https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html 

In [ ]:
df_dutch_umls.TTY.value_counts()

| TTY  | Description | Count | Example | Reference|
| - | - | - | - | - |
| PT | Designated preferred name| 111766 | harthypertrofie, Pancoast-syndroom ||
| LLT | Lower Level Term | 71603 | heupkombreuk, buikkramp| |
| LN | LOINC official fully specified name | 52313 | fencyclidine:massa/massa:moment:haar:kwantitatief | |
| MH | Main heading | 28657 | Dehydratie, Astma | |
| SY | Designated synonym | 11863 | Spanningshoofdpijn, Ziekte van Hodgkin | |
| OL | Non-current Lower Level Term| 9291 | acquired immunodeficiency syndrome, ankylose van gewricht, meerdere plaatsen | https://meddra.org/sites/default/files/page/documents_insert/meddra_-_terminologies_coding.pdf |
| HT | Hierarchical term | 3295 | calciummetabolismestoornissen, oculaire hemorragische aandoeningen	 | |
| LO | Obsolete official fully specified name | 1696| promyelocyten/100 leukocyten:getalsfractie:mom...	| |
| HG | High Level Group Term |  337| complicaties geassocieerd met medisch hulpmiddel, zuur-basestoornissen | |
| SMQ| Standardised MedDRA Query |  225| Leveraandoeningen (SMQ) , Tumormarkers (SMQ) | |
| CP | ICPC component process (in original form) |   38| Ander bloedonderzoek, Medicatie/recept/injectie | |
| OS | System-organ class |   27| Bloed- en lymfestelselaandoeningen, Infecties en parasitaire aandoeningen | |
| AB | Abbreviation in any source vocabulary |   27| Infec, Neopl, Ear, Endo | |

In [ ]:
# Select a set of TTYs that seem most relevant for entity linking
tty_selection = ['PT', 'LLT', 'MH', 'SY']
df_dutch_umls_subset = df_dutch_umls[df_dutch_umls.TTY.isin(tty_selection)].copy()

# Keep only relevant columns
df_dutch_umls_subset = df_dutch_umls_subset[['CUI', 'STR', 'TTY', 'SAB']]
df_dutch_umls_subset.rename({'CUI': 'cui', 'STR': 'str', 'TTY': 'tty', 'SAB': 'sab'}, inplace=True, axis=1)

# Most of the terms in UMLS have the Metathesaurus preferred name in English. 
# For a rough but effective fix to get a good preferred name for the Dutch terms, 
# change the terms that have the value "Designated preferred name" (PT) for the 
# Term Type in Source (TTY) to the Metathesaurus preferred name (PN). All others
# can be saved as synonym.
df_dutch_umls_subset.tty.replace({'PT': 'PN',
                                  'LLT': 'SY',
                                  'MH': 'SY'}, inplace=True)

# Remove "NAO" ("Niet Anders Omschreven"), which is relevant for the source terminlogy but not for entity linking.
# See https://meddra.org/sites/default/files/guidance/file/intguide_15_0_dutch.pdf
df_dutch_umls_subset.str = df_dutch_umls_subset.str.replace({' NAO': '', ' \(NAO\)': '', ' nao': ''}, regex=True)

# Sort values
df_dutch_umls_subset.sort_values(by=['cui', 'tty', 'str', 'sab'], inplace=True)

# Drop duplicates, only keep the first entry (which is a PN because we sorted)
print(f'Records before dropping duplicates: {df_dutch_umls_subset.shape[0]}')
df_dutch_umls_subset = df_dutch_umls_subset.drop_duplicates(subset=['cui', 'str'], keep='first').reset_index(drop=True)
print(f'Records after dropping duplicates: {df_dutch_umls_subset.shape[0]}')

# Because dropped duplicates, only the first value in SAB is saved. Because we lost the information of other colums, rename the values to UMLS-dutch
df_dutch_umls_subset['sab'] = 'UMLS-dutch'
df_dutch_umls_subset.head(20)

## Adding TUI (types)
UMLS concepts have one or multiple types. These types are kept in a separate table, `MRSTY`. See https://semanticnetwork.nlm.nih.gov/download/SemGroups.txt for all types.

In [ ]:
# Load TUI table from MySQL
query = """
SELECT cui, tui, sty FROM umls.mrsty
"""
df_tui = pd.read_sql_query(query, con=connection)

# Add TUI column to previously created dataframe
df_dutch_umls_subset_tui = df_dutch_umls_subset.merge(df_tui, how='left', on='cui')

# View some concepts that have multiple TUIs
df_dutch_umls_subset_tui[df_dutch_umls_subset_tui.duplicated(subset=['cui', 'str'], keep=False)].head(10)

In [ ]:
print(f'Number of unique TUIs in Dutch UMLS subset: {len(df_dutch_umls_subset_tui.tui.unique())}')
df_dutch_umls_subset_tui.sty.value_counts().head(10)

We could implement filtering of TUIs here. This depends on the domain and question of subsequent analysis. For SNOMED

In [ ]:
# Save UMLS 
df_dutch_umls_subset_tui.to_csv('04_ConceptDB/umls-dutch_v1.3-no-drugs.csv', index=False)

## Add SNOMED drug concepts
As mentioned in the introduction of this notebook, in this section drug concepts will be added to the UMLS concept table.

### Load SNOMED US data

In [ ]:
# Credentials to connect to remote MySQL database that contains SNOMEDCT-US.
# This is required for mapping SNOMED IDs from Dutch SNOMED drug concepts to UMLS IDs.
host = ''
username = 'root'
password = ''
port='3306'
database = 'umls'

In [ ]:
# Create the connection
connection_string = f'mysql://{username}:{password}@{host}:{port}/{database}'
connection = create_engine(connection_string)

# By removing the obsolute terms, we will solve some one-to-many mapping issues
query = """
SELECT distinct cui, scui FROM MRCONSO where sab = 'SNOMEDCT_US' and SUPPRESS !='O'
"""
df_snomed_us = pd.read_sql_query(query, con=connection)
df_snomed_us.scui = df_snomed_us.scui.astype(int)
print(f'SNOMED US terms with UMLS CUI: {df_snomed_us.shape[0]}')
df_snomed_us.head()

### Load SNOMED Dutch data

In [ ]:
# Read Dutch SNOMED concept table to retrieve drug names
df_snomed_dutch = pd.read_csv('04_ConceptDB/snomedct-dutch_v1.0.csv')
df_snomed_dutch.cui = df_snomed_dutch.cui.astype(int)
df_snomed_dutch_drugs = df_snomed_dutch[df_snomed_dutch.tui == 'substantie']
print(f'Number of SNOMED names from type "substantie": {df_snomed_dutch_drugs.shape[0]}')

In [ ]:
df_snomed_dutch_drugs_cuis = df_snomed_dutch[df_snomed_dutch.tui == 'substantie'].cui.unique()
print(f'Number of unique SNOMED CUIs from type "substantie" {len(df_snomed_dutch_drugs_cuis)}')

This means that on average, every SNOMED concept from type "substantie" has 1.8 names.

### Combine SNOMED Dutch with SNOMED US to add UMLS CUIs

In [ ]:
# Select only SNOMED US concepts that are in SNOMED Dutch "substance" subset
df_snomed_us_drugs = df_snomed_us[df_snomed_us.scui.isin(df_snomed_dutch_drugs_cuis)]
print(f'Number of SNOMED US concepts that are also present in SNOMED Dutch "substance" subset: {df_snomed_us_drugs.shape[0]}')
print(f'Number of unique SNOMED IDs in subset: {len(df_snomed_us_drugs.scui.unique())}')
print(f'Number of unique UMLS IDs in subset: {len(df_snomed_us_drugs.cui.unique())}')

This shows the many to many mapping. Some SNOMED IDs map to multiple UMLS IDs, while some UMLS IDs map to multiple SNOMED IDs. 

In [ ]:
# Visualize which SNOMED IDs map to multiple UMLS IDs
df_snomed_us_drugs_sorted = df_snomed_us_drugs.sort_values(['scui', 'cui'])
print(f'Number of SNOMED IDs that map to multiple UMLS IDs: {sum(df_snomed_us_drugs_sorted.duplicated(["scui"]))}')
df_snomed_us_drugs_sorted[df_snomed_us_drugs_sorted.duplicated(['scui'], keep=False)].head()

This means there are some SNOMED IDs that map to multiple UMLS CUIs

In [ ]:
# Visualize which SNOMED IDs map to multiple UMLS IDs including the Dutch name ('str')
df_snomed_dutch_drugs[df_snomed_dutch_drugs.cui.isin(df_snomed_us_drugs_sorted[df_snomed_us_drugs_sorted.duplicated(['scui'], keep=False)].scui)].head()

In [ ]:
# For 105 CUIs that map to multiple SNOMED IDs, we will take a random SNOMED ID by only keeping the first entry
df_snomed_us_drugs_unique = df_snomed_us_drugs_sorted.drop_duplicates(['scui'])
print(f'Number of SNOMED IDs that map to UMLS IDs after dropping duplicates: {df_snomed_us_drugs_unique.shape[0]}')

In [ ]:
# For a small amount of records, multiple SNOMED IDs map to the same CUI. 
# This is fine, it means that these UMLS CUIs will have more names from SNOMED.
len(df_snomed_us_drugs_unique.scui.unique()) - len(df_snomed_us_drugs_unique.cui.unique())

In [ ]:
# Merge SNOMED-UMLS table with SNOMED-Drugs table
df_umls_dutch_drugs = df_snomed_dutch_drugs.merge(df_snomed_us_drugs_unique, left_on='cui', right_on='scui', how='inner')
df_umls_dutch_drugs = df_umls_dutch_drugs[['cui_y', 'str', 'tty', 'sab']]
df_umls_dutch_drugs.rename({'cui_y': 'cui'}, inplace=True, axis=1)

print(f'Dutch SNOMED drug concepts with UMLS ID: {len(df_umls_dutch_drugs.cui.unique())}')
print(f'Dutch SNOMED drug names with UMLS ID: {df_umls_dutch_drugs.shape[0]}')

# Show example
df_umls_dutch_drugs[df_umls_dutch_drugs.cui=='C0025677']

### Merge Dutch UMLS with Dutch UMLS drugs

In [ ]:
print(f'Dutch UMLS concepts: {df_dutch_umls_subset_tui.shape[0]}')
print(f'Dutch UMLS concepts with SNOMED drug name: {df_umls_dutch_drugs.shape[0]}')
df_umls_dutch_total = pd.concat([df_dutch_umls_subset, df_umls_dutch_drugs])
print(f'Concatenated: {df_umls_dutch_total.shape[0]}')

In [ ]:
# Remove duplicate names, because a few drug names were already in our initial UMLS set
# We only keep the first, because we assume these concept already havea primary name (PN) in the initial set
df_umls_dutch_total.drop_duplicates(subset=['cui', 'str'], keep='first', inplace=True)
print(f'After removing duplicate names: {df_umls_dutch_total.shape[0]}')

In [ ]:
df_umls_dutch_total

### Add TUIs to merged table

In [ ]:
# Add TUIs
df_umls_dutch_total_with_tui = df_umls_dutch_total.merge(df_tui, how='left', on='cui')
print(f'Dutch UMLS names with SNOMED drug names with TUI: {df_umls_dutch_total_with_tui.shape[0]}')

In [ ]:
df_umls_dutch_total_with_tui.to_csv('04_ConceptDB/umls-dutch_v1.3-complete.csv', index=False)